<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Boston 데이터를 대상으로 모델링를 진행합니다.
- 개별 모델과 보팅(Voting) 모델의 관계를 확인합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [ ]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/Jangrae/csv/master/boston.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [ ]:
# 상위 몇 개 행 확인
data.head()

In [ ]:
# 하위 몇 개 행 확인
data.tail()

In [ ]:
# 변수 확인
data.info()

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) x, y 분리**

In [ ]:
# target 확인
target = 'medv'

# 데이터 분리
x = data.drop(columns=target)
y = data.loc[:, target]

**2) 학습용, 평가용 데이터 분리**

In [ ]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 우선 회귀 문제인지 분류 문제인지 명확히 구분합니다.

**1) 불러오기**

- 사용할 라이브러리를 모두 불러옵니다.

In [ ]:
# 불러오기
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import *

**2) 개별 모델 선언과 학습**

- 개별 모델을 선언하고 학습합니다.

In [ ]:
# Linear Regression
model_lr = LinearRegression()
model_lr.fit(x_train, y_train)

# KNN
model_knn = make_pipeline(MinMaxScaler(), KNeighborsRegressor())
model_knn.fit(x_train, y_train)

# Decision Tree
model_dt = DecisionTreeRegressor(random_state=1)
model_dt.fit(x_train, y_train)

# Random Forest
model_rdf = RandomForestRegressor(random_state=1)
model_rdf.fit(x_train, y_train)

# Light GBM
model_lgb = LGBMRegressor(random_state=1, verbose=-1)
model_lgb.fit(x_train, y_train)

**3) 예측값 평균 확인**

- 개별 모델의 예측값 평균을 확인합니다.

In [ ]:
# 예측 결과 수집
pred_dict = {'p1': model_lr.predict(x_test),
             'p2': model_knn.predict(x_test),
             'p3': model_dt.predict(x_test),
             'p4': model_rdf.predict(x_test),
             'p5': model_lgb.predict(x_test)}

# 데이터프레임 선언
result = pd.DataFrame(pred_dict)

# 평균 계산
result['mean'] = result.mean(axis=1)

# 확인
result.head(10)

**4) 보팅 모델 사용**

- 보팅 모델을 만들고 학습한 후 예측해봅니다.

In [ ]:
# 보팅 모델 선언
estimators = [('lr', model_lr), 
              ('knn', model_knn), 
              ('dt', model_dt), 
              ('rdf', model_rdf), 
              ('lgb', model_lgb)]

model = VotingRegressor(estimators=estimators)

# 학습하기
model.fit(x_train, y_train)

# 예측하기
y_pred = model.predict(x_test)

**5) 평균과 비교**

- 이전에 확인한 평균과 보팅 모델의 예측값을 비교합니다.

In [ ]:
# 평균 계산
result['y_pred'] = y_pred

# 확인
result.head(10)

**6) 성능 확인**

- 보팅 모델의 성능을 확인합니다.

In [ ]:
# 평가하기
print('* MAE:', mean_absolute_error(y_test, y_pred))
print('* R2:', r2_score(y_test, y_pred))

- 개별 모델 성능을 확인합니다.

In [ ]:
# 개별 모델 성능 확인
for m in [model_lr, model_knn, model_dt, model_rdf, model_lgb]:
    m.fit(x_train, y_train)
    y_pred = m.predict(x_test)
    print(f'* {m.__class__.__name__} R2 Score: {r2_score(y_test, y_pred):.4f}')

# 5.성능 튜닝

- GridSearchCV로 개별 모델의 성능을 최적화합니다.
- 성능이 최적화된 개별 모델을 사용하는 보팅 모델을 만들고 성능을 확인합니다.

**1) 개별 모델 성능 최적화**

- GridSearchCV로 개별 모델의 성능을 최적화합니다.

In [ ]:
# Linear Regression
model_lr = LinearRegression()
model_lr = model_lr.fit(x_train, y_train)

# KNN
param = {'kneighborsregressor__n_neighbors': range(2, 11, 1)}
model_grid = GridSearchCV(make_pipeline(MinMaxScaler(), KNeighborsRegressor()), param)
model_grid.fit(x_train, y_train)
model_knn = model_grid.best_estimator_

# Decision Tree
param = {'max_depth': range(1, 11, 1)}
model_grid = GridSearchCV(DecisionTreeRegressor(), param)
model_grid.fit(x_train, y_train)
model_dt = model_grid.best_estimator_

# Random Forest
param = {'max_depth': range(1, 11, 1)}
model_grid = GridSearchCV(RandomForestRegressor(), param)
model_grid.fit(x_train, y_train)
model_rdf = model_grid.best_estimator_

# LightGBM
param = {'max_depth': range(1, 11, 1)}
model_grid = GridSearchCV(LGBMRegressor(verbose=-1), param)
model_grid.fit(x_train, y_train)
model_lgb = model_grid.best_estimator_

**2) 보팅 모델 선언**

- 성능이 최적화된 개별 모델을 사용하는 보팅 모델을 만들고 성능을 확인합니다.

In [ ]:
# 보팅 모델 선언
estimators = [('lr', model_lr), 
              ('knn', model_knn), 
              ('dt', model_dt), 
              ('rdf', model_rdf), 
              ('lgb', model_lgb)]

model = VotingRegressor(estimators=estimators)

# 학습하기
model.fit(x_train, y_train)

# 예측하기
y_pred = model.predict(x_test)

# 평가하기
print('* MAE:', mean_absolute_error(y_test, y_pred))
print('* R2:', r2_score(y_test, y_pred))

- 개별 모델 성능도 확인합니다.

In [ ]:
# 개별 모델 성능 확인
for m in [model_lr, model_knn, model_dt, model_rdf, model_lgb]:
    y_pred = m.predict(x_test)
    print(f'* {m.__class__.__name__} R2 Score: {r2_score(y_test, y_pred):.4f}')